# Exercício de Self-Attention com PyTorch
Neste exercício, você irá codificar o mecanismo de Self-Attention do zero, com sugestões utilizando a biblioteca PyTorch.

Conforme visto em sala, o Self-attention é um componente central dos Transformers, as redes neurais que impulsionam os modelos de linguagem modernos como o BERT. Entender o self-attention é fundamental para compreender como esses modelos conseguem processar e interpretar a linguagem.

## Contexto
Um Transformer processa um texto de entrada, que é dividido em tokens. O Self-attention permite que o modelo determine a relação entre diferentes tokens em uma sequência.



# Dicas de ferramental

Para implementar será necessário saber realizar algumas operações. Como por exemplo inicializar pesos, multiplicação entre matrizes, calcular transposta etc. Vamos utilizar o framework `PyTorch` como sugestão, mas fica a cargo de cada um escolher a tecnologia que sinta mais à vontade.


## Multiplicação de matrizes

Podemos utilizar a função `torch.matmul()`. A função matmul pode ser utilizada para realizar a multiplicação de matrizes entre **q** e **k**. O resultado esperado, é uma matriz de pontuações que indica a similaridade de cada query com cada key. Uma pontuação alta significa que um token é muito similar a outro.




In [8]:
# Exemplo de multiplicação de matrizes
import torch

# Define as matrizes
matriz_A = torch.tensor([[1, 2], [3, 4]])
matriz_B = torch.tensor([[5, 6], [7, 8]])

# Multiplica as matrizes
torch.matmul(matriz_A, matriz_B)

tensor([[19, 22],
        [43, 50]])

## Transposta de uma matriz

Podemos utilizar o método `.transpose()` para calcular a transposta, conforme exemplo abaixo a seguir. A função do matriz.transpose(0, 1) inverte a primeira dimensão (índice 0, que são as linhas) com a segunda dimensão (índice 1, que são as colunas), resultando na matriz transposta.


In [9]:
# Define a matriz
matriz = torch.tensor([[1, 2, 3],
                       [4, 5, 6]])

# Transpõe a matriz
matriz.transpose(0, 1)

tensor([[1, 4],
        [2, 5],
        [3, 6]])

## Divisão de matrizes

Para efetuar o escalonamento das similaridades, evitando que valores muito grandes dominem o processo de `softmax`, será necessário efetuar uma divisão por um valor escalar calculado, conforme a fórmula vista na aula. Segue exemplo de como fazer com PyTorch abaixo:

In [10]:
# Define a matriz
matriz = torch.tensor([[10, 20], [30, 40]])
escalar = 2

# Divide a matriz por um escalar
matriz_dividida = matriz / escalar

matriz_dividida

tensor([[ 5., 10.],
        [15., 20.]])

## Aplicação de Softmax

Após o escalonamento, será necessário transformar em porcentagens de atenção as pontuações de similaridade obtidas. Para isso pode se utilizar a função `softmax`, que converte uma lista de números em uma distribuição de probabilidade. Ela garante que todos os valores somem 1. Cada valor na matriz de percentuais calculada representa o peso de atenção, ou seja, a porcentagem de "importância" que um token deve dar a todos os outros tokens da sequência.

O parâmetro `dim` diz ao PyTorch se essa conversão deve ser feita linha por linha, coluna por coluna ou ao longo de outra dimensão específica. Para nosso caso deve utilizar a dimensão da coluna.

```python
perc_atencao = torch.nn.functional.softmax(s_similiridades, dim=self.col_dim)
```


---

# Tarefas do Exercício
## 1. Codifique uma Classe Básica de Self-Attention
Sua primeira tarefa é completar a classe SelfAttention. Esta classe receberá as codificações de tokens como entrada e executará os cálculos de self-attention para produzir as pontuações de atenção. O método __init__ já está fornecido, mas você precisará preencher o método forward. Levando em consideração a equação vista:



$$Attention(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$



**Instruções**:

* Você trabalhará no método `forward` da classe `SelfAttention`.

* Crie os vetores de Queries, Keys e Values: Use as matrizes de pesos fornecidas (self.W_q, self.W_k e self.W_v) para transformar as token_encodings de entrada nos vetores de query (q), key (k) e value (v).

* Calcule as Pontuações de Similaridade: Compute as pontuações de similaridade realizando a multiplicação da matriz de queries (q) pela transposta das keys (k
T
 ).

* Dimensione as Pontuações: Dimensione as pontuações de similaridade dividindo-as pela raiz quadrada do número de colunas nas keys.

* Aplique o Softmax: Aplique a função softmax às pontuações dimensionadas para obter as porcentagens de atenção. Lembre-se de especificar a dimensão sobre a qual o softmax deve ser aplicado.

* Calcule as Pontuações Finais de Atenção: Multiplique as porcentagens de atenção pelos values (v) para obter as pontuações finais de atenção.

**Observações:**

-  Por motivos didáticos, não utilizaremos bias, nem faremos o treinamento dos pesos, nem trabalharemos com geração de embeddings. Portanto basta inicializar definindo um seed, conforme código de exemplo, para que possamos verificar se os cálculos estão sendo feitos corretamente com os valores de embeddings sugeridos.


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    """
    Uma classe básica de Self-Attention.
    """
    def __init__(self, d_model=2,
                 row_dim=0,
                 col_dim=1):
        super().__init__()

        # Inicializa os Pesos (W) para queries, keys e values
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = row_dim
        self.col_dim = col_dim

    def forward(self, token_encodings):
        """
        Executa o cálculo de self-attention.

        Args:
            token_encodings (torch.Tensor): Um tensor de entrada com as codificações dos tokens.
        Returns:
            torch.Tensor: O tensor com as pontuações de atenção.
        """
        # Criação de queries, keys e values
        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)

        # Computa as pontuações de similaridade
        sims = torch.matmul(q, k.transpose(-2, -1))

        # Dimensiona as similaridades
        d_k = k.size(-1)
        scaled_sims = sims / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

        # Aplica softmax para obter as porcentagens de atenção
        attention_percents = F.softmax(scaled_sims, dim=-1)

        # Calcula as pontuações finais de atenção
        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores


---

## 2. Teste sua Classe de Self-Attention
Após completar a classe SelfAttention, você a testará com alguns dados de exemplo.

**Instruções:**

Execute o código abaixo.

A saída do seu código deve corresponder à saída esperada.

In [12]:
# Cria uma matriz de codificações de token
encodings_matrix = torch.tensor([[1.16, 0.23],
                                 [0.57, 1.36],
                                 [4.41, -2.16]])

# Define a semente para reprodutibilidade
torch.manual_seed(42)

# Cria um objeto de self-attention
selfAttention = SelfAttention(d_model=2,
                               row_dim=0,
                               col_dim=1)

# Calcula as pontuações de atenção para as codificações de token
selfAttention(encodings_matrix)

tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)

**Saída Esperada:**



```
tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)
```





---

# 3. Implemente a Masked Self-Attention
Sua tarefa é modificar a classe `SelfAttention` anterior para incluir o comportamento de máscara criar uma nova classe `MaskedSelfAttention`.

## **Instruções:**

* A partir do código da sua classe `SelfAttention`, adicione uma lógica de mascaramento antes da função softmax.

* Você precisará criar uma máscara (mask) que tenha o mesmo formato da matriz de pontuações de similaridade (`sims`). Esta máscara deve conter zeros na diagonal principal e abaixo dela, e `-inf` (número muito grande e negativo) acima dela.

* Use a função `torch.triu()` para criar a parte superior da matriz de forma triangular.

* Adicione a máscara à matriz de pontuações de similaridade (`sims`). O valor `-inf` garantirá que o softmax transforme a pontuação de atenção em zero, ignorando os tokens futuros.

* Mantenha o restante da sua implementação de self-attention, como a escalabilidade e a multiplicação com a matriz de valores, intacto.

* Lembre-se que o "mascaramento" acontece ao fazer a adição da matriz de máscara, conforme a fórmula derivada a seguir:


$$\text{MaskedAttention}(Q, K, V) = \text{softmax}\left(\frac{QK^T + M}{\sqrt{d_k}}\right)V$$


In [13]:
class MaskedSelfAttention(nn.Module):
    """
    Uma classe básica de Masked Self-Attention.
    """
    def __init__(self, d_model=2,
                 row_dim=0,
                 col_dim=1):
        super().__init__()

        # Inicializa os Pesos (W) para queries, keys e values
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = row_dim
        self.col_dim = col_dim

    def forward(self, token_encodings):
        """
        Executa o cálculo de Masked Self-Attention.

        Args:
            token_encodings (torch.Tensor): Um tensor de entrada com as codificações dos tokens.
        Returns:
            torch.Tensor: Um tensor com as pontuações de atenção mascaradas.
        """
        # Criação de queries, keys e values
        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)

        # Computa as pontuações de similaridade
        sims = torch.matmul(q, k.transpose(-2, -1))

        # Pega o tamanho da sequência a partir da matriz de similaridades
        seq_len = sims.size(-1)

        # Cria uma máscara triangular superior
        mask = torch.triu(torch.ones(seq_len, seq_len, device=token_encodings.device), diagonal=1)

        # Cria a matriz M, substituindo os 1s por -infinito
        M = mask.masked_fill(mask == 1, float('-inf'))

        # Adiciona a máscara M às pontuações de similaridade
        masked_sims = sims + M

        # Dimensiona as similaridades
        d_k = k.size(-1)
        scaled_sims = masked_sims / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

        # Aplica softmax para obter as porcentagens de atenção
        attention_percents = F.softmax(scaled_sims, dim=-1)

        # Calcula as pontuações finais de atenção
        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores

In [14]:
# Cria uma matriz de codificações de token
encodings_matrix = torch.tensor([[1.16, 0.23],
                                 [0.57, 1.36],
                                 [4.41, -2.16]])

# Define a semente para reprodutibilidade
torch.manual_seed(42)

# Cria um objeto de self-attention
selfAttention = MaskedSelfAttention(d_model=2,
                               row_dim=0,
                               col_dim=1)

# Calcula as pontuações de atenção para as codificações de token
selfAttention(encodings_matrix)

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)